# MI test

In [1]:
import sys
import pandas as pd
import numpy as np
sys.path.append('/Users/paolo/Documents/Droughts/scripts')


In [2]:
from mixedRVMI import MIEstimate
from mixedRVMI import CMIEstimate

In [3]:
# importo pickle con il dataframe
# riduco a 100.000 dati per non essere troppo pesante 
# aggiungo una colonna non linearmente correlata e una indipendente
current_df = pd.read_pickle("/Users/paolo/Documents/Droughts/data/NDVI_2020052.pkl")
current_df = current_df.dropna(subset = ["SMN"])
current_df = current_df.iloc[0:100000]
current_df['SMN_2'] = current_df['SMN'].apply(lambda row: row*row)
current_df['SMN_ind'] = np.linspace(0,100,num=len(current_df['SMN']))
current_df.head()

,SMN,latitude,longitude,SMN_2,SMN_ind
97,0.003,71.514,-179.982,0.000009,0.000
98,0.001,71.478,-179.982,0.000001,0.001
99,0.001,71.442,-179.982,0.000001,0.002
100,0.012,71.406,-179.982,0.000144,0.003
101,0.012,71.370,-179.982,0.000144,0.004


In [5]:
# testo la MI
print("MI(SMN,SMN^2) = "+str(MIEstimate(current_df['SMN'].to_numpy(),current_df['SMN_2'].to_numpy(),estimate='digamma',k=3)))
print("MI(SMN,independent feature) = "+str(MIEstimate(current_df['SMN'].to_numpy(),current_df['SMN_ind'].to_numpy(),estimate='digamma',k=3)))
print("MI(SMN,both) = "+str(MIEstimate(current_df['SMN'].to_numpy(),current_df.loc[:,['SMN_2','SMN_ind']].to_numpy(),estimate='digamma',k=3)))

MI(SMN,SMN^2) = 4.946787949670897
MI(SMN,independent feature) = 0.9782831328718034
MI(SMN,both) = 1.1940409535125753


In [7]:
# testo la conditional
#k=300
print("MI(SMN,both) = "+str(MIEstimate(current_df['SMN'].to_numpy(),current_df.loc[:,['SMN_2','SMN_ind']].to_numpy(),estimate='digamma',k=300)))
print("CMI(SMN,SMN^2|independent_feature) = "+str(CMIEstimate(Y=current_df['SMN'].to_numpy(),X=current_df['SMN_2'].to_numpy(),Z=current_df['SMN_ind'].to_numpy(),k=300,estimate='digamma')))
print("CMI(SMN,independent feature|SMN^2) = "+str(CMIEstimate(Y=current_df['SMN'].to_numpy(),X=current_df['SMN_ind'].to_numpy(),Z=current_df['SMN_2'].to_numpy(),k=300,estimate='digamma')))


MI(SMN,both) = 0.017079894375836957
CMI(SMN,SMN^2|independent_feature) = 5.490683181450309e-05
CMI(SMN,independent feature|SMN^2) = -4.6060667698020055


In [8]:
# k=3
print("MI(SMN,both) = "+str(MIEstimate(current_df['SMN'].to_numpy(),current_df.loc[:,['SMN_2','SMN_ind']].to_numpy(),estimate='digamma',k=3)))
print("CMI(SMN,SMN^2|independent_feature) = "+str(CMIEstimate(Y=current_df['SMN'].to_numpy(),X=current_df['SMN_2'].to_numpy(),Z=current_df['SMN_ind'].to_numpy(),k=3,estimate='digamma')))
print("CMI(SMN,independent feature|SMN^2) = "+str(CMIEstimate(Y=current_df['SMN'].to_numpy(),X=current_df['SMN_ind'].to_numpy(),Z=current_df['SMN_2'].to_numpy(),k=3,estimate='digamma')))


MI(SMN,both) = 1.1940409535125753
CMI(SMN,SMN^2|independent_feature) = 0.21575782064077276
CMI(SMN,independent feature|SMN^2) = -3.752746996158321


In [6]:
# confronto con stimatore mi di scikit
from sklearn.feature_selection import mutual_info_regression
print("MI(SMN,SMN^2) = "+str(mutual_info_regression(current_df['SMN'].to_numpy().reshape(-1,1),current_df['SMN_2'].to_numpy().reshape(-1,1),n_neighbors=3)))
print("MI(SMN,independent feature) = "+str(mutual_info_regression(current_df['SMN'].to_numpy().reshape(-1,1),current_df['SMN_ind'].to_numpy().reshape(-1,1),n_neighbors=3)))
print("MI(SMN,both) = "+str(mutual_info_regression(current_df.loc[:,['SMN_2','SMN_ind']].to_numpy(),current_df['SMN'].to_numpy().reshape(-1,1),n_neighbors=3)))

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


MI(SMN,SMN^2) = [4.95054125]


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


MI(SMN,independent feature) = [0.50640791]


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


MI(SMN,both) = [4.94565152 0.5417765 ]
